In [ ]:
!pip install fpdf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from datetime import datetime
import os
from google.colab import drive
from google.colab import files
from fpdf import FPDF
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages



# Montar Google Drive
drive.mount('/content/drive')

# Ruta del archivo en Google Drive
ruta_drive = "/content/drive/My Drive/finanzas.csv"

# Cargar datos financieros desde un archivo CSV
def cargar_datos():
    global finanzas
    if os.path.exists(ruta_drive):
        finanzas = pd.read_csv(ruta_drive, parse_dates=["Fecha"])
    else:
        print("Archivo no encontrado. Creando un nuevo archivo 'finanzas.csv' en Google Drive.")
        finanzas = pd.DataFrame(columns=["Fecha", "Tipo", "Categoría", "Monto"])
        guardar_datos()

# Guardar datos en el archivo CSV en Google Drive
def guardar_datos():
    finanzas.to_csv(ruta_drive, index=False)
    print("Datos guardados correctamente en Google Drive.\n")

# Descargar archivo CSV
def descargar_csv():
    if os.path.exists(ruta_drive):
        files.download(ruta_drive)
        print("El archivo se ha descargado correctamente.")
    else:
        print("Error: El archivo 'finanzas.csv' no existe en Google Drive.")

# Registrar ingresos y gastos con validaciones adicionales
def registrar_movimiento():
    # Selección del tipo de movimiento
    print("\nSeleccione el tipo de movimiento:")
    print("1. Ingreso")
    print("2. Gasto")
    tipo_opcion = input("Ingrese el número correspondiente: ").strip()

    if tipo_opcion == "1":
        tipo = "ingreso"
    elif tipo_opcion == "2":
        tipo = "gasto"
    else:
        print("Opción no válida. Intente nuevamente.")
        return

    # Selección de categoría
    print("\nSeleccione la categoría:")
    categorias = ["Alimentación", "Transporte", "Entretenimiento", "Otros"]
    for i, categoria in enumerate(categorias, 1):
        print(f"{i}. {categoria}")
    try:
        categoria_opcion = int(input("Ingrese el número correspondiente: ").strip())
        if 1 <= categoria_opcion <= len(categorias):
            categoria = categorias[categoria_opcion - 1]
        else:
            print("Opción no válida. Intente nuevamente.")
            return
    except ValueError:
        print("Entrada no válida. Debe ser un número. Intente nuevamente.")
        return

    # Validación de monto
    try:
        monto = float(input("Ingrese el monto: "))
        if monto <= 0:
            print("Error: El monto debe ser un número positivo.")
            return
    except ValueError:
        print("Error: Entrada de monto inválida. Intente nuevamente con un número.")
        return

    # Validación de fecha
    try:
        fecha = input("Ingrese la fecha (YYYY-MM-DD): ")
        fecha = pd.to_datetime(fecha, format="%Y-%m-%d", errors="raise")
    except ValueError:
        print("Error: Formato de fecha inválido. Use el formato YYYY-MM-DD.")
        return

    # Actualización del DataFrame
    global finanzas
    nuevo_registro = pd.DataFrame([{
        "Fecha": fecha,
        "Tipo": tipo,
        "Categoría": categoria,
        "Monto": monto
    }])

    if finanzas.empty:
        finanzas = nuevo_registro
    else:
        finanzas = pd.concat([finanzas, nuevo_registro], ignore_index=True)

    guardar_datos()
    print("\nRegistro añadido con éxito.")


# Función para registrar un salario
def registrar_salario():
    try:
        monto = float(input("Ingrese el monto del salario: "))
        if monto <= 0:
            print("Error: El monto del salario debe ser un número positivo.")
            return
    except ValueError:
        print("Error: Entrada de monto inválida. Intente nuevamente con un número.")
        return

    # Validación de fecha
    try:
        fecha = input("Ingrese la fecha de recepción del salario (YYYY-MM-DD): ")
        fecha = pd.to_datetime(fecha, format="%Y-%m-%d", errors="raise")
    except ValueError:
        print("Error: Formato de fecha inválido. Use el formato YYYY-MM-DD.")
        return

    global finanzas
    nuevo_registro = pd.DataFrame([{
        "Fecha": fecha,
        "Tipo": "ingreso",
        "Categoría": "Salario",
        "Monto": monto
    }])

    if finanzas.empty:
        finanzas = nuevo_registro
    else:
        finanzas = pd.concat([finanzas, nuevo_registro], ignore_index=True)

    guardar_datos()
    print("\nSalario registrado con éxito.")

#Registra los gastos mensuales
def registrar_gastos_mensuales():
    try:
        monto = float(input("Ingrese el monto total de los gastos mensuales: "))
        if monto <= 0:
            print("Error: El monto debe ser un número positivo.")
            return
    except ValueError:
        print("Error: Entrada de monto inválida. Intente nuevamente con un número.")
        return

    # Validación de fecha
    try:
        fecha = input("Ingrese la fecha del mes correspondiente (YYYY-MM-DD): ")
        fecha = pd.to_datetime(fecha, format="%Y-%m-%d", errors="raise")
    except ValueError:
        print("Error: Formato de fecha inválido. Use el formato YYYY-MM-DD.")
        return

    global finanzas
    nuevo_registro = pd.DataFrame([{
        "Fecha": fecha,
        "Tipo": "gasto",
        "Categoría": "Gastos Mensuales",
        "Monto": monto
    }])

    if finanzas.empty:
        finanzas = nuevo_registro
    else:
        finanzas = pd.concat([finanzas, nuevo_registro], ignore_index=True)

    guardar_datos()
    print("\nGastos mensuales registrados con éxito.")



# Mostrar gráficos con opciones y agrupación por día
import matplotlib.dates as mdates

def mostrar_graficos():
    categoria = input("¿Desea ver el desglose por categoría? (si/no): ").strip().lower()
    if categoria == "si":
        finanzas_gastos = finanzas[finanzas["Tipo"] == "gasto"]
        if finanzas_gastos.empty:
            print("No hay datos de gastos para mostrar.")
            return
        finanzas_gastos.groupby("Categoría")["Monto"].sum().plot(kind="bar", figsize=(10, 6))
        plt.title("Gastos por Categoría")
        plt.xlabel("Categoría")
        plt.ylabel("Monto")
        plt.show()
    elif categoria == "no":
        if finanzas.empty:
            print("No hay datos para mostrar en el historial.")
            return
        try:
            # Crear DataFrames separados para ingresos y gastos
            finanzas_ingresos = finanzas[finanzas["Tipo"] == "ingreso"]
            finanzas_gastos = finanzas[finanzas["Tipo"] == "gasto"]

            # Configurar el formato de fechas
            date_format = mdates.DateFormatter("%Y-%m-%d")

            # Graficar puntos para ingresos y gastos
            plt.figure(figsize=(10, 6))
            ax = plt.gca()  # Obtener los ejes actuales
            ax.xaxis.set_major_formatter(date_format)  # Aplicar el formato de fechas

            if not finanzas_ingresos.empty:
                plt.scatter(finanzas_ingresos["Fecha"], finanzas_ingresos["Monto"], color="green", label="Ingresos")
            if not finanzas_gastos.empty:
                plt.scatter(finanzas_gastos["Fecha"], finanzas_gastos["Monto"], color="red", label="Gastos")

            plt.title("Historial de Ingresos y Gastos")
            plt.xlabel("Fecha")
            plt.ylabel("Monto")
            plt.xticks(rotation=45)  # Rotar etiquetas para mayor legibilidad
            plt.legend()
            plt.tight_layout()
            plt.show()
        except Exception as e:
            print(f"Error al mostrar el gráfico: {e}")
    else:
        print("Opción no válida, intente de nuevo.")


# Predicción de gastos con validación de meses
def predecir_gastos():
    try:
        meses = int(input("¿Cuántos meses desea predecir? (número): "))
        if meses <= 0:
            print("Error: El número de meses debe ser positivo.")
            return

        # Filtrar y procesar los datos de gastos
        finanzas_gastos = finanzas[finanzas["Tipo"] == "gasto"].set_index("Fecha")
        finanzas_gastos = finanzas_gastos.resample("M").sum()["Monto"]

        if finanzas_gastos.empty:
            print("No hay datos suficientes de gastos para realizar una predicción.")
            return

        # Filtrar y calcular el promedio de salario mensual
        salarios = finanzas[(finanzas["Tipo"] == "ingreso") & (finanzas["Categoría"] == "Salario")]
        salario_promedio = salarios["Monto"].mean() if not salarios.empty else 0

        if salario_promedio == 0:
            print("No se han registrado salarios, no se puede estimar un límite de gasto.")
        else:
            print(f"Salario mensual promedio registrado: ${salario_promedio:.2f}")

        # Entrenar el modelo ARIMA
        modelo = ARIMA(finanzas_gastos, order=(1, 1, 1))
        modelo_fit = modelo.fit()
        pred = modelo_fit.forecast(steps=meses)

        print("\nPredicción de gastos para los próximos meses:")
        print(pred)

        # Calcular el porcentaje de gasto promedio respecto al salario
        gasto_promedio_mensual = finanzas_gastos.mean()
        porcentaje_gasto_salario = (gasto_promedio_mensual / salario_promedio) * 100 if salario_promedio > 0 else 0
        print(f"Porcentaje promedio de gasto mensual respecto al salario: {porcentaje_gasto_salario:.2f}%")

        # Mostrar predicción y límite de gasto basado en salario
        plt.figure(figsize=(10, 6))
        pred.plot(kind="bar", label="Predicción de Gastos", color="orange")

        # Mostrar línea de referencia de salario mensual
        plt.axhline(y=salario_promedio, color="blue", linestyle="--", label="Salario Promedio")
        plt.axhline(y=(salario_promedio * (porcentaje_gasto_salario / 100)), color="red", linestyle="--", label="Límite de Gasto Sugerido")

        plt.title("Predicción de Gastos Futuros con Referencia de Salario")
        plt.xlabel("Mes")
        plt.ylabel("Monto")
        plt.legend()
        plt.show()

    except ValueError:
        print("Entrada inválida para el número de meses. Intente de nuevo.")
    except Exception as e:
        print(f"Error en la predicción: {e}")

# Mostrar resumen financiero
def mostrar_resumen():
    # Mostrar resumen global
    ingresos_totales = finanzas[finanzas["Tipo"] == "ingreso"]["Monto"].sum()
    gastos_totales = finanzas[finanzas["Tipo"] == "gasto"]["Monto"].sum()
    ahorro_neto = ingresos_totales - gastos_totales

    print("\n--- Resumen Financiero Global ---")
    print(f"Total Ingresos: ${ingresos_totales:.2f}")
    print(f"Total Gastos: ${gastos_totales:.2f}")
    print(f"Ahorro Neto: ${ahorro_neto:.2f}")
    if ingresos_totales > 0:
        print(f"Porcentaje del ingreso gastado: {gastos_totales / ingresos_totales * 100:.2f}%")
        print(f"Porcentaje del ingreso ahorrado: {ahorro_neto / ingresos_totales * 100:.2f}%")
    else:
        print("No se han registrado ingresos, no es posible calcular porcentajes.")

    # Mostrar desglose por categoría
    print("\n--- Desglose por Categoría ---")
    if finanzas.empty:
        print("No hay datos financieros registrados.")
        return

    resumen_categorias = finanzas.groupby(["Tipo", "Categoría"])["Monto"].sum().reset_index()
    for tipo in resumen_categorias["Tipo"].unique():
        print(f"\n{tipo.capitalize()}s:")
        tipo_data = resumen_categorias[resumen_categorias["Tipo"] == tipo]
        for _, row in tipo_data.iterrows():
            print(f"  - {row['Categoría']}: ${row['Monto']:.2f}")

    # Rango de fechas
    print("\n--- Resumen por Rango de Fechas ---")
    try:
        fecha_inicio = input("Ingrese la fecha de inicio (YYYY-MM-DD): ")
        fecha_inicio = pd.to_datetime(fecha_inicio, format="%Y-%m-%d", errors="raise")
        fecha_fin = input("Ingrese la fecha de fin (YYYY-MM-DD): ")
        fecha_fin = pd.to_datetime(fecha_fin, format="%Y-%m-%d", errors="raise")
        finanzas_rango = finanzas[(finanzas["Fecha"] >= fecha_inicio) & (finanzas["Fecha"] <= fecha_fin)]
        ingresos_rango = finanzas_rango[finanzas_rango["Tipo"] == "ingreso"]["Monto"].sum()
        gastos_rango = finanzas_rango[finanzas_rango["Tipo"] == "gasto"]["Monto"].sum()
        ahorro_rango = ingresos_rango - gastos_rango

        print(f"\nResumen del {fecha_inicio.date()} al {fecha_fin.date()}:")
        print(f"  Total Ingresos: ${ingresos_rango:.2f}")
        print(f"  Total Gastos: ${gastos_rango:.2f}")
        print(f"  Ahorro Neto: ${ahorro_rango:.2f}")
    except ValueError:
        print("Error: Formato de fecha inválido o rango no especificado correctamente.")


def borrar_datos():
    global finanzas
    finanzas = pd.DataFrame(columns=["Fecha", "Tipo", "Categoría", "Monto"])
    guardar_datos()
    print("Todos los datos han sido borrados.\n")


#Generar PDF
def generar_resumen_pdf():
    # Crear un objeto PDF
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.add_page()
    pdf.set_font("Arial", size=12)

    # Título
    pdf.set_font("Arial", size=16, style="B")
    pdf.cell(200, 10, "Resumen Financiero", ln=True, align="C")
    pdf.ln(10)

    # Resumen Global
    ingresos_totales = finanzas[finanzas["Tipo"] == "ingreso"]["Monto"].sum()
    gastos_totales = finanzas[finanzas["Tipo"] == "gasto"]["Monto"].sum()
    ahorro_neto = ingresos_totales - gastos_totales

    pdf.set_font("Arial", size=12)
    pdf.cell(200, 10, "--- Resumen Financiero Global ---", ln=True)
    pdf.cell(200, 10, f"Total Ingresos: ${ingresos_totales:.2f}", ln=True)
    pdf.cell(200, 10, f"Total Gastos: ${gastos_totales:.2f}", ln=True)
    pdf.cell(200, 10, f"Ahorro Neto: ${ahorro_neto:.2f}", ln=True)

    if ingresos_totales > 0:
        pdf.cell(200, 10, f"Porcentaje del ingreso gastado: {gastos_totales / ingresos_totales * 100:.2f}%", ln=True)
        pdf.cell(200, 10, f"Porcentaje del ingreso ahorrado: {ahorro_neto / ingresos_totales * 100:.2f}%", ln=True)
    else:
        pdf.cell(200, 10, "No se han registrado ingresos para calcular porcentajes.", ln=True)
    pdf.ln(10)

    # Resumen por Categorías
    pdf.cell(200, 10, "--- Desglose por Categoría ---", ln=True)
    resumen_categorias = finanzas.groupby(["Tipo", "Categoría"])["Monto"].sum().reset_index()
    for tipo in resumen_categorias["Tipo"].unique():
        pdf.cell(200, 10, f"{tipo.capitalize()}s:", ln=True)
        tipo_data = resumen_categorias[resumen_categorias["Tipo"] == tipo]
        for _, row in tipo_data.iterrows():
            pdf.cell(200, 10, f"  - {row['Categoría']}: ${row['Monto']:.2f}", ln=True)
    pdf.ln(10)

    # Gráficas en un archivo temporal
    pdf.cell(200, 10, "--- Gráficas ---", ln=True)

    # Crear y guardar las gráficas en un archivo temporal
    grafica_path = "/content/grafica_resumen.pdf"
    with PdfPages(grafica_path) as pdf_pages:
        # Gráfica de desglose por categoría
        finanzas_gastos = finanzas[finanzas["Tipo"] == "gasto"]
        if not finanzas_gastos.empty:
            plt.figure(figsize=(10, 6))
            finanzas_gastos.groupby("Categoría")["Monto"].sum().plot(kind="bar", title="Gastos por Categoría")
            plt.xlabel("Categoría")
            plt.ylabel("Monto")
            plt.tight_layout()
            pdf_pages.savefig()
            plt.close()

        # Gráfica de puntos de ingresos y gastos
        plt.figure(figsize=(10, 6))
        ingresos = finanzas[finanzas["Tipo"] == "ingreso"]
        gastos = finanzas[finanzas["Tipo"] == "gasto"]
        if not ingresos.empty:
            plt.scatter(ingresos["Fecha"], ingresos["Monto"], color="green", label="Ingresos")
        if not gastos.empty:
            plt.scatter(gastos["Fecha"], gastos["Monto"], color="red", label="Gastos")
        plt.title("Historial de Ingresos y Gastos")
        plt.xlabel("Fecha")
        plt.ylabel("Monto")
        plt.legend()
        plt.tight_layout()
        pdf_pages.savefig()
        plt.close()

    # Guardar el PDF
    resumen_path = "/content/resumen_financiero.pdf"
    pdf.output(resumen_path)
    print("Resumen financiero generado correctamente.")

    # Descargar archivos
    from google.colab import files
    files.download(resumen_path)
    files.download(grafica_path)


# Menú principal con opciones y validaciones
def menu():
    cargar_datos()
    while True:
        print("\n--- Sistema de Gestión Financiera ---")
        print("1. Registrar Ingreso/Gasto")
        print("2. Registrar Salario")
        print("3. Registrar Gastos Mensuales")
        print("4. Ver Gráficos")
        print("5. Predecir Gastos")
        print("6. Mostrar Resumen Financiero")
        print("7. Descargar Resumen PDF con Gráficas")  # Nueva opción
        print("8. Borrar Todos los Datos")
        print("9. Descargar Archivo CSV")
        print("10. Salir")

        opcion = input("Seleccione una opción: ")

        if opcion == "1":
            registrar_movimiento()
        elif opcion == "2":
            registrar_salario()
        elif opcion == "3":
            registrar_gastos_mensuales()
        elif opcion == "4":
            mostrar_graficos()
        elif opcion == "5":
            predecir_gastos()
        elif opcion == "6":
            mostrar_resumen()
        elif opcion == "7":  # Nueva opción para generar PDF
            generar_resumen_pdf()
        elif opcion == "8":
            borrar_datos()
        elif opcion == "9":
            descargar_csv()
        elif opcion == "10":
            print("Saliendo del sistema...")
            break
        else:
            print("Opción no válida, intente de nuevo.")

# Ejecutar el menú
menu()

  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=0e9041da2d599b37a14f3c2a5088fb3e1fa8321c5e575796757ce6d70045ab95
  Stored in directory: /root/.cache/pip/wheels/f9/95/ba/f418094659025eb9611f17cbcaf2334236bf39a0c3453ea455
Successfully built fpdf
Mounted at /content/drive

--- Sistema de Gestión Financiera ---
1. Registrar Ingreso/Gasto
2. Registrar Salario
3. Registrar Gastos Mensuales
4. Ver Gráficos
5. Predecir Gastos
6. Mostrar Resumen Financiero
7. Descargar Resumen PDF con Gráficas
8. Borrar Todos los Datos
9. Descargar Archivo CSV
10. Salir
